In [ ]:

import csv
import os
import sys
from datetime import datetime

CACHE_FILE = 'cache.csv'
ORIGINAL_FILE = 'names.csv'
CURRENT_YEAR = datetime.now().year

def load_file(filepath):
    if not os.path.exists(filepath):
        print(f"Fehler: Datei '{filepath}' nicht gefunden.")
        return None, None

    with open(filepath, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        rows = [[cell.strip() for cell in row] for row in reader]

    if not rows:
        print(f"Datei '{filepath}' ist leer.")
        return None, None

    header = rows[0]
    data = rows[1:]
    return header, data

def write_cache(header, rows, filepath=CACHE_FILE):
    with open(filepath, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(rows)

def filter_by_column(header, rows, column, allowed_values):
    if column not in header:
        print(f"Spalte '{column}' nicht gefunden.")
        return []
    idx = header.index(column)
    return [row for row in rows if row[idx] in allowed_values]

def sort_by_column(header, rows, column, reverse=False):
    if column not in header:
        print(f"Spalte '{column}' nicht gefunden.")
        return rows
    idx = header.index(column)
    try:
        rows.sort(key=lambda r: int(r[idx]) if r[idx].isdigit() else r[idx], reverse=reverse)
    except Exception:
        rows.sort(key=lambda r: r[idx], reverse=reverse)
    return rows

def count_total(header, rows):
    if 'Number' not in header:
        return 0
    idx = header.index('Number')
    total = 0
    for r in rows:
        if r[idx].isdigit():
            total += int(r[idx])
    return total

def average_age(header, rows):
    if 'YearOfBirth' not in header:
        return None
    idx = header.index('YearOfBirth')
    ages = []
    for r in rows:
        if r[idx].isdigit():
            ages.append(CURRENT_YEAR - int(r[idx]))
    return sum(ages)/len(ages) if ages else None

def print_table(header, rows, limit=None):
    if not rows:
        print("Keine Daten zum Anzeigen.")
        return
    col_widths = [max(len(h), max((len(str(r[i])) for r in rows), default=0)) + 2 for i, h in enumerate(header)]
    sep_line = '+' + '+'.join('-' * w for w in col_widths) + '+'
    print(sep_line)
    print('|' + '|'.join(f" {h.ljust(col_widths[i]-2)} " for i, h in enumerate(header)) + '|')
    print(sep_line)
    for row in (rows[:limit] if limit else rows):
        print('|' + '|'.join(f" {str(row[i]).ljust(col_widths[i]-2)} " for i in range(len(header))) + '|')
    print(sep_line)

def main():
    header, data = load_file(ORIGINAL_FILE)
    if header is None:
        sys.exit(1)
    write_cache(header, data)
    current_header, current_rows = header, data

    while True:
        print("\nMenü:")
        print(" 1 = Filtern")
        print(" 2 = Sortieren")
        print(" 3 = Statistik")
        print(" 4 = Anzeigen")
        print(" 5 = Speichern")
        print(" 0 = Beenden")
        choice = input("Auswahl: ").strip()

        if choice == '1':
            col = input("Spalte zum Filtern: ").strip()
            vals = input("Erlaubte Werte (Komma-getrennt): ").split(',')
            vals = [v.strip() for v in vals]
            current_rows = filter_by_column(current_header, current_rows, col, vals)
            write_cache(current_header, current_rows)
            print(f"Gefiltert nach {col} in {vals}: {len(current_rows)} Zeilen übrig.")

        elif choice == '2':
            col = input("Spalte zum Sortieren: ").strip()
            asc = input("Aufsteigend sortieren? (j/n): ").strip().lower() == 'j'
            current_rows = sort_by_column(current_header, current_rows, col, reverse=not asc)
            write_cache(current_header, current_rows)
            print(f"Sortiert nach {col}, {'aufsteigend' if asc else 'absteigend'}.")

        elif choice == '3':
            total = count_total(current_header, current_rows)
            avg = average_age(current_header, current_rows)
            print(f"Gesamtanzahl (Number): {total}")
            print(f"Durchschnittsalter (YearOfBirth): {avg:.1f} Jahre" if avg else "Keine Altersdaten vorhanden.")

        elif choice == '4':
            lim = input("Maximale Zeilenzahl anzeigen (Enter = alle): ").strip()
            lim_val = int(lim) if lim.isdigit() else None
            print_table(current_header, current_rows, limit=lim_val)

        elif choice == '5':
            fn = input("Dateiname für Export (.csv): ").strip() or 'export.csv'
            write_cache(current_header, current_rows, filepath=fn)
            print(f"Ergebnis in Datei '{fn}' gespeichert.")

        elif choice == '0':
            print("Programm beendet. Auf Wiedersehen!")
            break

        else:
            print("Ungültige Eingabe. Bitte wähle eine Zahl zwischen 0 und 5.")

if __name__ == '__main__':
    main()
